# 📈 Forecasting de ventas 2025
Este notebook importa las mismas librerías que el notebook de entrenamiento y carga el archivo de inferencia para realizar análisis y predicciones.

## 1. Importar librerías necesarias
Se importan las mismas librerías que en el notebook de entrenamiento: pandas, numpy, matplotlib, seaborn, scikit-learn, streamlit y holidays.

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

## 2. Cargar archivo ventas_2025_inferencia en un DataFrame
Se carga el archivo '../data/raw/inferencia/ventas_2025_inferencia.csv' en un DataFrame llamado inferencia_df.

In [43]:
inferencia_df = pd.read_csv('../data/raw/inferencia/ventas_2025_inferencia.csv')
print('Primeras filas del DataFrame de inferencia:')
inferencia_df.head()

Primeras filas del DataFrame de inferencia:


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


## 🔄 TRANSFORMACIÓN DE DATOS PARA INFERENCIA

Aplicaremos exactamente las mismas transformaciones que en el notebook de entrenamiento para que el dataframe tenga la misma estructura.

### 3. Conversión de fecha a datetime

In [44]:
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])
print('Fecha convertida a datetime')
print(inferencia_df['fecha'].dtype)

Fecha convertida a datetime
datetime64[ns]


### 4. Variables temporales y de calendario

In [45]:
# Variables temporales básicas
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['mes_nombre'] = inferencia_df['fecha'].dt.month_name()
inferencia_df['dia'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek
inferencia_df['nombre_dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['fin_de_semana'] = inferencia_df['dia_semana'].isin([5, 6])

# Festivos en España
years = inferencia_df['año'].unique()
es_holidays = holidays.country_holidays('ES', years=years)
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)
inferencia_df['nombre_festivo'] = inferencia_df['fecha'].map(lambda x: es_holidays.get(x, None))

# Black Friday (último viernes de noviembre)
def es_blackfriday(fecha):
    if fecha.month == 11 and fecha.weekday() == 4:
        next_friday = fecha + pd.Timedelta(days=7)
        return next_friday.month != 11
    return False
inferencia_df['es_blackfriday'] = inferencia_df['fecha'].apply(es_blackfriday)

# Cyber Monday (primer lunes después de Black Friday)
def es_cybermonday(fecha):
    if fecha.month == 11 and fecha.weekday() == 0:
        bf = [d for d in inferencia_df['fecha'] if es_blackfriday(d) and d.year == fecha.year]
        if len(bf) > 0:
            bf_date = max(bf)
            return fecha > bf_date and (fecha - bf_date).days <= 3
    return False
inferencia_df['es_cybermonday'] = inferencia_df['fecha'].apply(es_cybermonday)

# Día laborable
inferencia_df['es_laborable'] = (~inferencia_df['es_festivo']) & (~inferencia_df['fin_de_semana'])

# Semana del año
inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week

# Trimestre
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter

# Inicio/fin de mes
inferencia_df['inicio_mes'] = inferencia_df['dia'] <= 3
inferencia_df['fin_mes'] = inferencia_df['dia'] >= (inferencia_df['fecha'] + pd.offsets.MonthEnd(0)).dt.day - 2

print('Variables temporales y de calendario creadas')
print(f'Shape: {inferencia_df.shape}')

C:\Users\PC\AppData\Local\Temp\ipykernel_15108\2326633075.py:13: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)


Variables temporales y de calendario creadas
Shape: (888, 29)


### 5. Creación de lags (1-7) y media móvil de 7 días usando valores históricos

In [46]:
# IMPORTANTE: Para inferencia, necesitamos los valores históricos de 2024
# Cargaremos el df histórico para obtener los lags correctos

# Cargar el dataframe histórico procesado
df_historico = pd.read_csv('../data/processed/df.csv')
df_historico['fecha'] = pd.to_datetime(df_historico['fecha'])

# Función para crear lags y media móvil
lags = range(1, 8)

def crear_lags_media(df):
    df = df.sort_values('fecha')
    for lag in lags:
        df[f'unidades_vendidas_lag{lag}'] = df['unidades_vendidas'].shift(lag)
    df['unidades_vendidas_mm7'] = df['unidades_vendidas'].rolling(window=7).mean()
    return df

# Concatenar datos históricos con inferencia para calcular lags correctamente
# Usamos datos de 2024 y 2025 del histórico como base
df_base_historico = df_historico[df_historico['año'].isin([2024, 2025])].copy()

# Aplicar por producto
lag_dfs = []
for producto_id in inferencia_df['producto_id'].unique():
    # Datos históricos del producto
    hist_producto = df_base_historico[df_base_historico['producto_id'] == producto_id][['fecha', 'producto_id', 'unidades_vendidas']].copy()
    
    # Datos de inferencia del producto
    inf_producto = inferencia_df[inferencia_df['producto_id'] == producto_id].copy()
    
    # Concatenar histórico + inferencia
    combined = pd.concat([hist_producto, inf_producto[['fecha', 'producto_id', 'unidades_vendidas']]])
    combined = combined.sort_values('fecha')
    
    # Crear lags sobre los datos combinados
    combined = crear_lags_media(combined)
    
    # Filtrar solo las filas de inferencia
    inf_con_lags = combined[combined['fecha'].isin(inf_producto['fecha'])]
    
    # Hacer merge con el resto de columnas de inferencia
    inf_producto_completo = inf_producto.merge(
        inf_con_lags[['fecha', 'producto_id'] + [f'unidades_vendidas_lag{i}' for i in lags] + ['unidades_vendidas_mm7']],
        on=['fecha', 'producto_id'],
        how='left'
    )
    
    lag_dfs.append(inf_producto_completo)

inferencia_df = pd.concat(lag_dfs)

# NO eliminamos registros con nulos - los mantenemos para noviembre
cols_lag_mm = [f'unidades_vendidas_lag{i}' for i in lags] + ['unidades_vendidas_mm7']

print('Variables de lags y media móvil creadas usando datos históricos')
print(f'Total de registros: {len(inferencia_df)}')
print(f'Registros con nulos en lags: {inferencia_df[cols_lag_mm].isnull().any(axis=1).sum()}')
print(inferencia_df[['fecha', 'producto_id', 'unidades_vendidas'] + cols_lag_mm].head(10))

Variables de lags y media móvil creadas usando datos históricos
Total de registros: 888
Registros con nulos en lags: 720
       fecha producto_id  unidades_vendidas  unidades_vendidas_lag1  \
0 2025-10-25    PROD_001               26.0                    80.0   
1 2025-10-26    PROD_001               20.0                    26.0   
2 2025-10-27    PROD_001               16.0                    20.0   
3 2025-10-28    PROD_001               15.0                    16.0   
4 2025-10-29    PROD_001               14.0                    15.0   
5 2025-10-30    PROD_001               10.0                    14.0   
6 2025-10-31    PROD_001               14.0                    10.0   
7 2025-11-01    PROD_001                NaN                    14.0   
8 2025-11-02    PROD_001                NaN                     NaN   
9 2025-11-03    PROD_001                NaN                     NaN   

   unidades_vendidas_lag2  unidades_vendidas_lag3  unidades_vendidas_lag4  \
0                   

### 6. Variable de descuento porcentaje

In [47]:
inferencia_df['descuento_porcentaje'] = (inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base'] * 100

print('Variable descuento_porcentaje creada')
print(inferencia_df[['fecha', 'producto_id', 'precio_base', 'precio_venta', 'descuento_porcentaje']].head())

Variable descuento_porcentaje creada
       fecha producto_id  precio_base  precio_venta  descuento_porcentaje
0 2025-10-25    PROD_001          115        113.13             -1.626087
1 2025-10-26    PROD_001          115        105.75             -8.043478
2 2025-10-27    PROD_001          115        114.95             -0.043478
3 2025-10-28    PROD_001          115        117.31              2.008696
4 2025-10-29    PROD_001          115        108.10             -6.000000


### 7. Variables precio_competencia y ratio_precio

In [48]:
competidores = ['Amazon', 'Decathlon', 'Deporvillage']
inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# Eliminar las columnas de competidores
inferencia_df = inferencia_df.drop(columns=competidores)

print('Variables precio_competencia y ratio_precio creadas')
print('Columnas de competidores eliminadas')
print(inferencia_df[['fecha', 'producto_id', 'precio_venta', 'precio_competencia', 'ratio_precio']].head())

Variables precio_competencia y ratio_precio creadas
Columnas de competidores eliminadas
       fecha producto_id  precio_venta  precio_competencia  ratio_precio
0 2025-10-25    PROD_001        113.13          102.573333      1.102918
1 2025-10-26    PROD_001        105.75           98.356667      1.075169
2 2025-10-27    PROD_001        114.95           97.740000      1.176079
3 2025-10-28    PROD_001        117.31          103.146667      1.137313
4 2025-10-29    PROD_001        108.10          100.520000      1.075408


### 8. One-hot encoding y alineación de columnas con el dataframe histórico

In [49]:
# Crear copias con sufijo _h
for col in ['nombre', 'categoria', 'subcategoria']:
    inferencia_df[f'{col}_h'] = inferencia_df[col]

# One hot encoding
variables_h = ['nombre_h', 'categoria_h', 'subcategoria_h']
inferencia_df = pd.get_dummies(inferencia_df, columns=variables_h, prefix=variables_h)

# Alineación de columnas con el dataframe histórico
# El df_historico que cargamos es el básico sin lags ni one-hot
# Necesitamos obtener todas las columnas one-hot que existen en el entrenamiento
# Para eso, obtenemos las columnas del entrenamiento del histórico después del one-hot

# Obtener todas las columnas one-hot posibles del histórico
columnas_onehot_historico = [col for col in df_historico.columns if col.startswith(('nombre_h_', 'categoria_h_', 'subcategoria_h_'))]

# Añadir columnas one-hot faltantes con valor False
for col in columnas_onehot_historico:
    if col not in inferencia_df.columns:
        inferencia_df[col] = False

print('One-hot encoding aplicado y columnas alineadas con el histórico')
print(f'Shape final: {inferencia_df.shape}')
print(f'Columnas: {len(inferencia_df.columns)}')
print(f'Productos únicos: {inferencia_df["producto_id"].nunique()}')
inferencia_df.head()

One-hot encoding aplicado y columnas alineadas con el histórico
Shape final: (888, 81)
Columnas: 81
Productos únicos: 24


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,...,False,False,False,False,False,False,False,False,True,False
1,2025-10-26,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,20.0,105.75,2115.00,...,False,False,False,False,False,False,False,False,True,False
2,2025-10-27,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,16.0,114.95,1839.20,...,False,False,False,False,False,False,False,False,True,False
3,2025-10-28,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,15.0,117.31,1759.65,...,False,False,False,False,False,False,False,False,True,False
4,2025-10-29,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,14.0,108.10,1513.40,...,False,False,False,False,False,False,False,False,True,False


## 🎯 FILTRADO Y GUARDADO DE DATOS

### 9. Filtrado de solo noviembre (elimina octubre)

In [50]:
# Filtrar solo noviembre 2025 (eliminar octubre 2025)
registros_antes = len(inferencia_df)
inferencia_df = inferencia_df[inferencia_df['mes'] == 11]
registros_despues = len(inferencia_df)

print(f'✅ Registros de octubre eliminados: {registros_antes - registros_despues}')
print(f'📊 Registros restantes (solo noviembre 2025): {registros_despues}')
print(f'📅 Fechas: {inferencia_df["fecha"].min()} a {inferencia_df["fecha"].max()}')
print(f'🏷️ Productos únicos: {inferencia_df["producto_id"].nunique()}')
print(f'\n📋 Distribución de registros por producto:')
print(inferencia_df.groupby('producto_id').size().describe())

✅ Registros de octubre eliminados: 168
📊 Registros restantes (solo noviembre 2025): 720
📅 Fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
🏷️ Productos únicos: 24

📋 Distribución de registros por producto:
count    24.0
mean     30.0
std       0.0
min      30.0
25%      30.0
50%      30.0
75%      30.0
max      30.0
dtype: float64


In [51]:
inferencia_df.shape

(720, 81)

### 10. Guardado del dataframe transformado

In [52]:
ruta_guardado = '../data/processed/inferencia_df_transformado.csv'
inferencia_df.to_csv(ruta_guardado, index=False)

print(f'✅ DataFrame transformado guardado exitosamente en: {ruta_guardado}')
print(f'📊 Total de registros guardados: {len(inferencia_df)}')
print(f'📋 Total de columnas: {len(inferencia_df.columns)}')
print(f'\n🎯 El dataframe está listo para hacer predicciones con el modelo')
print(f'\nPrimeras filas del dataframe final:')
inferencia_df.head()

✅ DataFrame transformado guardado exitosamente en: ../data/processed/inferencia_df_transformado.csv
📊 Total de registros guardados: 720
📋 Total de columnas: 81

🎯 El dataframe está listo para hacer predicciones con el modelo

Primeras filas del dataframe final:


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
7,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
8,2025-11-02,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
9,2025-11-03,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
10,2025-11-04,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
11,2025-11-05,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
